## Fake News Classifier Using LSTM

In this project, I would try to implement LSTM (Long short-term memory) to classify fake news based on the `title` of the news. LSTM is an artificial neural network used in the field of deep learning and is heavily used in NLP based tasks.

The dataset has been taken from kaggle: https://www.kaggle.com/c/fake-news/data#

In [1]:
# Importing Libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout

In [2]:
# Checking tensorflow version
tf.__version__

'2.4.1'

In [3]:
# Read the dataset
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop Nan Values
df=df.dropna()
df.reset_index(inplace=True, drop = True)

In [5]:
# Get the Independent Features
X=df.drop('label',axis=1)

In [6]:
# Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
# Vocabulary size (size of the vocabulary dictionary)
voc_size=5000

### One-hot Representation

The first step here would be to convert the collection of words into a one-hot representation.
But before that, we need to make sure the data is pre-processed.

In [10]:
# creating a copy of X
messages=X.copy()

In [11]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/badvendetta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# Dataset Preprocessing

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # removing unnecessary characters
    review = review.lower() # converting to lower case
    review = review.split() # removing whitespaces
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # stemming the words
    review = ' '.join(review)
    corpus.append(review)

In [14]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [15]:
# Finally, converting to one-hot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4148, 1300, 4624, 2015, 1892, 2303, 3159, 4298, 701, 3989],
 [4578, 4961, 1510, 3672, 257, 2396, 4040],
 [1140, 4854, 399, 1007],
 [1001, 3746, 4782, 1268, 1819, 998],
 [1180, 257, 2025, 2471, 909, 1811, 257, 2530, 2121, 3782],
 [3513,
  628,
  4687,
  4061,
  3923,
  1217,
  2224,
  531,
  4501,
  2670,
  2202,
  4612,
  4737,
  4413,
  4040],
 [1321, 743, 4330, 4913, 4603, 348, 1430, 1330, 442, 411, 1259],
 [2787, 2603, 3043, 712, 4230, 4541, 1217, 2053, 442, 411, 1259],
 [3166, 3026, 3520, 39, 2686, 3487, 3849, 4049, 1217, 3054],
 [487, 4976, 4133, 25, 894, 4940, 3606, 804],
 [860, 4956, 1930, 2356, 3701, 2665, 1415, 830, 127, 757, 1290],
 [1268, 1187, 1892, 3487, 1217, 4230],
 [2510, 4140, 4693, 3776, 952, 4911, 4809, 97, 4387],
 [73, 4147, 825, 3417, 1014, 1516, 128, 442, 411, 1259],
 [2568, 1235, 668, 232, 4224, 442, 411, 1259],
 [4690, 2654, 1804, 1548, 130, 843, 3468, 3149, 2275, 3232],
 [4651, 2635, 4961],
 [121, 3651, 45, 1625, 1217, 3994, 4979, 4040],
 [677, 2262, 1510, 42

### Embedding Representation

In [16]:
sent_length=20 # max length of sentence we are allowing
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) # pre-padding used here
print(embedded_docs)

[[   0    0    0 ... 4298  701 3989]
 [   0    0    0 ...  257 2396 4040]
 [   0    0    0 ... 4854  399 1007]
 ...
 [   0    0    0 ...  442  411 1259]
 [   0    0    0 ... 2122 2851 2012]
 [   0    0    0 ... 3590 3465 1458]]


In [17]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4148,
       1300, 4624, 2015, 1892, 2303, 3159, 4298,  701, 3989], dtype=int32)

### Building the model

In [18]:
embedding_vector_features=40 # Number of features in the Feature representation
model=Sequential() # Building a sequential model
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # Adding an embedding layer
model.add(LSTM(100)) # Adding an LSTM layer with 100 neurons
model.add(Dense(1,activation='sigmoid')) # Adding a dense layer with sigmoid activation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) # Compiling the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
len(embedded_docs),y.shape

(18285, (18285,))

In [20]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [21]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

### Model Training

In [23]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 21ms/step - loss: 0.4823 - accuracy: 0.7603 - val_loss: 0.2000 - val_accuracy: 0.9110
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1415 - accuracy: 0.9460 - val_loss: 0.2025 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0959 - accuracy: 0.9659 - val_loss: 0.2209 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0616 - accuracy: 0.9788 - val_loss: 0.3172 - val_accuracy: 0.8971
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0365 - accuracy: 0.9885 - val_loss: 0.3160 - val_accuracy: 0.9104
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0200 - accuracy: 0.9946 - val_loss: 0.4277 - val_accuracy: 0.9104
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0200 - accuracy: 0.9949 - val_loss: 0.4589 - val_accuracy: 0.9047

### Performance Metrics And Accuracy

In [24]:
y_pred=model.predict_classes(X_test)

/Users/badvendetta/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
accuracy_score(y_test,y_pred)

0.8969345484672743

In [26]:
confusion_matrix(y_test,y_pred)

array([[3161,  258],
       [ 364, 2252]])

### Adding Dropout 

In [27]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(LSTM(100))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 24ms/step - loss: 0.5016 - accuracy: 0.7293 - val_loss: 0.2004 - val_accuracy: 0.9100
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1562 - accuracy: 0.9391 - val_loss: 0.1969 - val_accuracy: 0.9171
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1073 - accuracy: 0.9592 - val_loss: 0.2064 - val_accuracy: 0.9150
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0846 - accuracy: 0.9711 - val_loss: 0.2280 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0614 - accuracy: 0.9793 - val_loss: 0.2390 - val_accuracy: 0.9074
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0496 - accuracy: 0.9839 - val_loss: 0.2927 - val_accuracy: 0.9067
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0331 - accuracy: 0.9893 - val_loss: 0.3318 - val_accuracy: 0.9105

In [29]:
y_pred=model.predict_classes(X_test)

accuracy_score(y_test,y_pred)

0.9096934548467275

In [30]:
confusion_matrix(y_test,y_pred)

array([[3109,  310],
       [ 235, 2381]])